# Linear Regression

## Reading in the data

We'll be working with a dataset from Capital Bikeshare.

In [ ]:
# read the data and set the datetime as the index
import pandas as pd
bikes = pd.read_csv('bikeshare.csv', index_col='datetime', parse_dates=True)
print bikes.head(10)

## Renaming our outcome variable

We're going to rename one of the columns which is called "count" because "count" is already a function in Pandas for dataframes/

So we'll call it "total", and the inplace=True makes the rename overwrite the existing variable

In [ ]:
# "count" is a method, so it's best to name that column something else
bikes.rename(columns={'count':'total'}, inplace=True)

## Visualizing the data

Before building our regression model, we probably want to see which variables already have a strong linear relationship with total bikes rented.

We can examine how the variables are correlated with each other using the corr() function on the dataframe

In [ ]:
# the correlation matrix (ranges from 1 to -1)
#the row and column intersection shows the correlation between the two variables
#the diagonals are all 1.0 because a variable is perfectly with itself
bikes.corr()

We can visualize the correlation matrix using a heatmap where red indicate a strong positive correlation, blue indicates a strong negative correlation, and yellow indicates no correlation between two variables

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# visualize correlation matrix using a heatmap
#change the font size and figure size if you have problem reading the labels
plt.rcParams['figure.figsize'] = (40, 40)
plt.rcParams['font.size'] = 23


fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(bikes.corr())
fig.colorbar(cax)
ax.set_xticks(range(len(bikes.corr().index)))
ax.set_yticks(range(len(bikes.corr().index)))
ax.set_xticklabels(bikes.corr().index)
ax.set_yticklabels(bikes.corr().index)

plt.show()

Which variables look to be the most strongly correlated with "total"?

## Scatterplots

We can also visualize the data points of two variables as a scatterplot, which shows us how well a straight line fits the data

Here, we plot the the relationship of temperature and total bikes used at a specific time.

Feel free to enter your own predictor of interest in place of temperature to see  the relationship.

In [ ]:
# We can call the pandas internal scatterplot function
#the alpha =.2 makes the dots transparent to a degree, so you can see where there is a lot overlapping data
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
bikes.plot(kind='scatter', x='temp', y='total', alpha=0.2)

## Form of linear regression

Linear regression models take the following equational form:

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$

- $y$ is the response
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for $x_1$ (the first feature)
- $\beta_n$ is the coefficient for $x_n$ (the nth feature)

The $\beta$ values are called the **model coefficients**:

- These values are estimated (or "learned") during the model fitting process using the **least squares criterion**.
- Specifically, we are find the line (mathematically) which minimizes the **sum of squared residuals** (or "sum of squared errors").
- And once we've learned these coefficients, we can use the model to predict the response.

![Estimating coefficients](estimating_coefficients.png)

In the diagram above:

- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the vertical distances between the observed values and the least squares line.
- The optimization algorithm is trying to find the weights for each predictor variable in your model so that when you add them together, you produce a line that minimize the vertical distance between all of the points and the line

## Building a linear regression model

We are going to build a simple (one predictor) linear regression model, where temperature (in Celsius) tries to predict the total number of bikes checked out at a given hour.

First, let's define what features(predictors) we want to use.

For this example, the only feature we are using is 'temp', but if you wanted to use more, you could add them to the feature_cols list making sure to separate the name of each predictor with a comma.

After that, we are going to set our outcome variable (total) to be equal to y

In [ ]:
# create X and y
feature_cols = ['temp']
X = bikes[feature_cols]
y = bikes['total']

Now, let's build a model by importing the LinearRegression function from sklearn.

After we import it, we will create a LinearRegression object that we can use to fit a model where X can predict y

In [ ]:
# import, instantiate, fit
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X, y)

Now that out model is fit, we can see what the results are.

Specifically, we can see what the intercept (the value of x, temperature, when y, Total # of bike checked out, is 0) is
                                             
We can also see what the coefficient (weight) for temperature is

In [ ]:
# print the coefficients
print linreg.intercept_
print linreg.coef_

Interpreting the **intercept** ($\beta_0$):

- It is the value of $y$  when $x$=0.
- Thus, it is the estimated number of rentals when the temperature is 0 degrees Celsius.
- Therefore, we would expect 6.75 rentals on a given hour when it is 0 degrees Celsisus
- **Note:** It does not always make sense to interpret the intercept. Sometimes, a value of 0 for a predictor is impossible or unlikely to ever be observed (like  human height)

Interpreting the **"temp" coefficient** ($\beta_1$):

- It is the change in $y$ divided by change in $x$, or the "slope".
- Thus, a temperature increase of 1 degree Celsius is **associated with** a rental increase of 9.12 bikes.
- This is not a statement of causation.
- $\beta_1$ would be **negative** if an increase in temperature was associated with a **decrease** in rentals.

## Using the model for prediction

Now that we have a model fit to the data and we know the intercept and coefficients, we can predict the total number of bikes being checked out per hour if we know the temperature

If we wanted to predict how many bikes would be checked out when the temperature was 25 degrees Celsius, we would write:

In [ ]:
# manually calculate the prediction
linreg.intercept_ + linreg.coef_*25

We can also use the "predict" function for the linreg object:

In [ ]:
# use the predict method
linreg.predict(25)

## Does the scale of the features matter?

Let's say that temperature was measured in Fahrenheit, rather than Celsius. How would that affect the model?

Here is what out model looked before using celsius to predict total number of bikes checked out.

In [ ]:
plt.scatter(X, y,  color='blue',alpha=.2)
plt.plot(X, linreg.predict(X), color='red',linewidth=3)

In [ ]:
# create a new column for Fahrenheit temperature
bikes['temp_F'] = bikes['temp'] * 1.8 + 32
bikes.head()

In [ ]:
# create X and y
feature_cols = ['temp_F']
X = bikes[feature_cols]
y = bikes['total']

# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

# print the coefficients
print linreg.intercept_
print linreg.coef_

In [ ]:
# convert 25 degrees Celsius to Fahrenheit
25 * 1.8 + 32

In [ ]:
# predict rentals for 77 degrees Fahrenheit
linreg.predict(77)

In [ ]:
plt.scatter(X, y,  color='blue',alpha=.2)
plt.plot(X, linreg.predict(X), color='red',linewidth=3)

**Conclusion:** The prediction is exactly the same and the best fitting line is exactly the same. Threfore, the scale of the features is **irrelevant** for linear regression models. When changing the scale, we simply change our **interpretation** of the coefficients.

We don't need the temperature in Farenheit because it is providing redudant information, so we are going to drop it from the dataframe

In [ ]:
# remove the temp_F column
bikes.drop('temp_F', axis=1, inplace=True)
print bikes.head(5)

## Visualizing the data (part 2)

Let's start adding more predictors into the model.

In [ ]:
# explore more features
feature_cols = ['temp', 'season', 'weather', 'humidity']

In [ ]:
# multiple scatter plots in Pandas
fig, axs = plt.subplots(1, len(feature_cols), sharey=True)
for index, feature in enumerate(feature_cols):
    bikes.plot(kind='scatter', x=feature, y='total', ax=axs[index], figsize=(16, 3))

Are you seeing anything that you did not expect?

This is how season is coded with respect to month

In [ ]:
# cross-tabulation of season and month
pd.crosstab(bikes['season'], bikes.index.month)

In [ ]:
# box plot of rentals, grouped by season
bikes.boxplot(column='total', by='season')

Note that doesn't seem like a straight line captures the relationship between season (1,2,3,4) and number of rentals. Therefore, we want our model to take into account that season probably varies curvilinearly with total number of rentals.

## Curvilinear Relationships

If our data has a curvilinear relationship with one of the variables and the outcome, we can modify our linear regression regression equation to include a squared/polynomial/quadratic term.

$y = \beta_0 + \beta_1x_1 + \beta_2x_1^2$

- $y$ is the response
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for $x_1$ (the first feature)
- $\beta_n$ is the polynomial coefficient for $x_1$ (the first feature)

This equation produces a parabola rather than a straight line, can take into account a variable that we say predicts our outcome like a curve.

To account for these polynomial terms, we simply have to make a new variable that is the square the variable we think has a curvlinear relationship with our outcome. We can then enter it into the model as another variable, making sure to include the original variable as well. 


In [ ]:
bikes['season_squared'] = bikes['season']**2
feature_cols = ['season', 'season_squared']
X = bikes[feature_cols]
y = bikes['total']

# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

# print the coefficients
print linreg.intercept_
print linreg.coef_

plt.scatter(bikes['season'], y,  color='blue',alpha=.2)
plt.scatter(bikes['season'], linreg.predict(X), color='red')


Notice how the predicted points follow a curve, rather a straight line, as season increases from 1 to 4

## Interactions

Another kind of transformation you can do to your data is take into account the fact that the relationship between one variable and another depends on the level of a third variable.

Take the relationship between temperature and bike rentals. It is positive and a fairly strong effec (*r* = .39)

However, you can imagine that while hotter days do tend to get more people biking, it really depends on how humid it is.

That is, the relationship between temperature and bikes rented should be weaker if it is humid outside. A humid AND hot days is no good!

However, the humidity is low, then the relationship between temperature and bikes used should be stronger because you are now only considering days when it's not humid.

We can show this by looking at the correlation between temperature and total bikes rented, and then seeing how the correlations differ when we are below median humidity and above median humidity.

In [ ]:
median_humidity = bikes.humidity.median()
print np.corrcoef(bikes.temp,bikes.total)[0,1]
print np.corrcoef(bikes[bikes['humidity'] < median_humidity]['temp'], bikes[bikes['humidity'] < median_humidity]['total'])[0,1]
print np.corrcoef(bikes[bikes['humidity'] > median_humidity]['temp'], bikes[bikes['humidity'] > median_humidity]['total'])[0,1]


We can also plot this difference visually.

Notice how the relaitonship between temperature and total bikes rented is flatter (weaker) when humidity is high (the red line) vs. when humidity is low (the magenta line)

In [ ]:
feature_cols = ['temp']

X_lowhumidity = bikes[bikes['humidity'] < bikes.humidity.median()][feature_cols]
X_highhumidity = bikes[bikes['humidity'] > bikes.humidity.median()][feature_cols]
y_lowhumidity = bikes[bikes['humidity'] < bikes.humidity.median()]['total']
y_highhumidity =bikes[bikes['humidity'] > bikes.humidity.median()]['total']


linreg_lowhumidity = LinearRegression()
linreg_lowhumidity.fit(X_noholiday, y_noholiday)

linreg_highhumidity = LinearRegression()
linreg_highhumidity.fit(X_highhumidity, y_highhumidity)


plt.scatter(bikes['temp'], y,  color='blue',alpha=.2)
plt.scatter(X_lowhumidity, linreg_lowhumidity.predict(X_lowhumidity), color='magenta')
plt.scatter(X_highhumidity, linreg_highhumidity.predict(X_highhumidity), color='red')




To create a term that takes this interaction into account, you want to multiply the variables for temperature and humidity to make a temp X humid variable.

You would then enter this interaction variable as a feature in your model, and your regression would then take it into account.

Their coefficient though is not directly interpretable though

In [ ]:
bikes['temp_by_humid'] = bikes['temp'] * bikes['humid']

## Handling categorical features

scikit-learn expects all features to be numeric. So how do we include a categorical feature in our model?

- **Ordered categories:** transform them to sensible numeric values (example: small=1, medium=2, large=3)
- **Unordered categories:** use dummy encoding (0/1)

What are the categorical features in our dataset?

- **Ordered categories:** weather (already encoded with sensible numeric values)
- **Unordered categories:** season (needs dummy encoding), holiday (already dummy encoded), workingday (already dummy encoded)

For season, we can't simply leave the encoding as 1 = spring, 2 = summer, 3 = fall, and 4 = winter, because that would imply an **ordered relationship**. Instead, we create **multiple dummy variables:**

In [ ]:
# create dummy variables
season_dummies = pd.get_dummies(bikes['season'], prefix='season')

# print 5 random rows
season_dummies.sample(n=5, random_state=1)

However, we actually only need **three dummy variables (not four)**, and thus we'll drop the first dummy variable.

Why? Because three dummies captures all of the "information" about the season feature, and implicitly defines spring (season 1) as the **baseline level:**

In [ ]:
# drop the first column
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)

# print 5 random rows
season_dummies.sample(n=5, random_state=1)

In general, if you have a categorical feature with **k possible values**, you create **k-1 dummy variables**.

If that's confusing, think about why we only need one dummy variable for holiday, not two dummy variables (holiday_yes and holiday_no).

In [ ]:
# concatenate the original DataFrame and the dummy DataFrame (axis=0 means rows, axis=1 means columns)
bikes = pd.concat([bikes, season_dummies], axis=1)

# print 5 random rows
bikes.sample(n=5, random_state=1)

In [ ]:
# include dummy variables for season in the model
feature_cols = ['temp', 'season_2', 'season_3', 'season_4', 'humidity']
X = bikes[feature_cols]
y = bikes['total']
linreg = LinearRegression()
linreg.fit(X, y)
zip(feature_cols, linreg.coef_)

How do we interpret the season coefficients? They are **measured against the baseline (spring)**:

- Holding all other features fixed, **summer** is associated with a **rental decrease of 3.39 bikes** compared to the spring.
- Holding all other features fixed, **fall** is associated with a **rental decrease of 41.7 bikes** compared to the spring.
- Holding all other features fixed, **winter** is associated with a **rental increase of 64.4 bikes** compared to the spring.

Would it matter if we changed which season was defined as the baseline?

- No, it would simply change our **interpretation** of the coefficients.

**Important:** Dummy encoding is relevant for all machine learning models, not just linear regression models.

## Adding more features to the model

In [ ]:
# create a list of features
feature_cols = ['temp', 'season', 'weather', 'humidity']

In [ ]:
# create X and y
X = bikes[feature_cols]
y = bikes['total']

# instantiate and fit
linreg = LinearRegression()
linreg.fit(X, y)

# print the coefficients
print linreg.intercept_
print linreg.coef_

In [ ]:
# pair the feature names with the coefficients
zip(feature_cols, linreg.coef_)

Interpreting the coefficients:

- Holding all other features fixed, a 1 unit increase in **temperature** is associated with a **rental increase of 7.86 bikes**.
- Holding all other features fixed, a 1 unit increase in **season** is associated with a **rental increase of 22.5 bikes**.
- Holding all other features fixed, a 1 unit increase in **weather** is associated with a **rental increase of 6.67 bikes**.
- Holding all other features fixed, a 1 unit increase in **humidity** is associated with a **rental decrease of 3.12 bikes**.

Does anything look incorrect?

## Feature selection

How do we choose which features to include in the model? 

Why not use of **p-values** or **R-squared** for feature selection?

- Linear models rely upon **a lot of assumptions** (such as the features being independent), and if those assumptions are violated, p-values and R-squared are less reliable.
- Features that are unrelated to the response can still have **significant p-values**.
- Adding features to your model that are unrelated to the response will always **increase the R-squared value**, and adjusted R-squared does not sufficiently account for this.
- p-values and R-squared are **proxies** for our goal of generalization, whereas our ultimate goal is to is estimate performance and errors

More generally:

- There are different methodologies that can be used for solving any given data science problem, and this course follows a **machine learning methodology**.
- This course focuses on **general purpose approaches** that can be applied to any model, rather than model-specific approaches.

## Evaluation metrics for regression problems

Evaluation metrics for classification problems, such as **accuracy**, are not useful for regression problems. We need evaluation metrics designed for comparing **continuous values**.

Here are three common evaluation metrics for regression problems:

**Mean Absolute Error** (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$

**Mean Squared Error** (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$

**Root Mean Squared Error** (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$

Imagine that our true values for the outcome were 10, 7, 5, and 5
And that our regression line predicted 8, 6, 5 and 10

In [ ]:
# example true and predicted response values
true = [10, 7, 5, 5]
pred = [8, 6, 5, 10]

In [ ]:
# calculate these metrics by hand!
from sklearn import metrics
import numpy as np
print 'MAE:', metrics.mean_absolute_error(true, pred)
print 'MSE:', metrics.mean_squared_error(true, pred)
print 'RMSE:', np.sqrt(metrics.mean_squared_error(true, pred))

Comparing these metrics:

- **MAE** is the easiest to understand, because it's the average error.
- **MSE** is more popular than MAE, because MSE "punishes" larger errors (squaring the error makes large errors even larger), which tends to be useful in the real world.
- **RMSE** is even more popular than MSE, because unlike MSE, RMSE is interpretable in the "y" units.

All of these are **loss functions**, because we want to <u>minimize them</u>.

Here's an additional example, to demonstrate how MSE/RMSE punish larger errors. Notice how in this example, you are only off once, but when you are off, it is REALLY off:

<u>**AGAIN, SMALLER ERRORS ARE WHAT WE WANT. IDEALLY, YOU WANT AN RMSEA OF 0**</u>

In [ ]:
# same true values as above
true = [10, 7, 5, 5]

# new set of predicted values
pred = [10, 7, 5, 13]

# MAE is the same as before
print 'MAE:', metrics.mean_absolute_error(true, pred)

# MSE and RMSE are larger than before
print 'MSE:', metrics.mean_squared_error(true, pred)
print 'RMSE:', np.sqrt(metrics.mean_squared_error(true, pred))

## Comparing models

We can compare models by looking at their RMSEA and seeing which one is smaller. Let's compare the model with season vs. the model with season and its polynomial variable. Which one seems to produce less error / more accurate predictions?

In [ ]:
feature_cols = ['season']
X = bikes[feature_cols]
y = bikes['total']

linreg = LinearRegression()
linreg.fit(X, y)
y_pred = linreg.predict(X)
print "RMSE: %f" % np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
feature_cols = ['season', "season_squared"]
X = bikes[feature_cols]
y = bikes['total']

linreg = LinearRegression()
linreg.fit(X, y)
y_pred = linreg.predict(X)
print "RMSE: %f" % np.sqrt(metrics.mean_squared_error(y, y_pred))

## Try it yourself

Try to come up with the best performing model you can. Transform or make new variables in the data however you see fit.

Because in the real world, you would not know how many casual or registered users would use bikes on that day, you are not allowed to use 'casual' or 'registered' as predictors.

Because you are probably going to call the same commands over an over again, you can create a function that executes those commands if you give it the features (as a list) that you want to use.

In [ ]:
def testmodel(feature_cols):
    X = bikes[feature_cols]
    y = bikes['total']

    # instantiate and fit
    linreg = LinearRegression()
    linreg.fit(X, y)
    y_pred = linreg.predict(X)
    return np.sqrt(metrics.mean_squared_error(y, y_pred)) 


In [ ]:
###Hint: If you have a list of ideas, save them into a common list, 
#and run a for loop trying each one. Then update a dictionary that holds the lowest score and the feature associated with it 









## Overview of linear regression compared to other models

Advantages of linear regression:

- Simple to explain
- Highly interpretable
- Model training and prediction are fast
- No tuning is required (excluding regularization)
- Features don't need scaling
- Can perform well with a small number of observations
- Well-understood

Disadvantages of linear regression:

- Presumes a linear relationship between the features and the response
- Performance is (generally) not competitive with the best supervised learning methods due to high bias
- Can't automatically learn feature interactions